In [1]:
! pip3 uninstall -y opencv-contrib-python opencv-python
! pip3 uninstall -y opencv-python

! pip3 install imutils opencv-contrib-python
! apt install -y libgl1-mesa-glx

Found existing installation: opencv-contrib-python 4.5.1.48
Uninstalling opencv-contrib-python-4.5.1.48:
  Successfully uninstalled opencv-contrib-python-4.5.1.48
  Using cached opencv_contrib_python-4.5.1.48-cp36-cp36m-manylinux2014_x86_64.whl (56.3 MB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-glx is already the newest version (20.0.8-0ubuntu1~18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [5]:
! apt -y remove python-opencv

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package 'python-opencv' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [2]:
#config
import fiftyone as fo
import os
# import the necessary packages
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import imagenet_utils
from imutils.object_detection import non_max_suppression
import numpy as np
import random
import time
import cv2
import matplotlib.pyplot as plt
import imutils
import numpy as np


file_path = "/tf/testing"

model_path = '/tf/dataset/plane-detector'
INPUT_SIZE = (299, 299)
MIN_CONF = 1
FAST = True
batch_size = 1000
# Create the dataset
try:
    dataset = fo.load_dataset("plane-dataset")
except fo.core.dataset.DoesNotExistError:
    dataset = fo.Dataset(name="plane-dataset")
    

ServiceListenTimeout: fiftyone.core.service.DatabaseService failed to bind to port

In [7]:
# load our network weights from disk
print("[INFO] loading network...")
# load the trained model
model = load_model(model_path)
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)



[INFO] loading network...


In [8]:
def selective_search(image, fast):
	# initialize OpenCV's selective search implementation and set the
	# input image
	ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
	ss.setBaseImage(image)
	# check to see if we are using the *fast* but *less accurate* version
	# of selective search
	if fast:
		ss.switchToSelectiveSearchFast()
	# otherwise we are using the *slower* but *more accurate* version
	else:
		ss.switchToSelectiveSearchQuality()
	# run selective search on the input image
	rects = ss.process()
	# return the region proposal bounding boxes
	return rects

In [10]:
def findPlane(file_path):
    # load the input image
    image = cv2.imread(file_path)
    (H, W) = image.shape[:2]
    rects = selective_search(image, True)
    print("[INFO] {} regions found by selective search".format(len(rects)))
    # initialize the list of region proposals that we'll be classifying
    # along with their associated bounding boxes
    
    proposals = []
    boxes = []
    # loop over the region proposal bounding box coordinates generated by
    # running selective search
    for (x, y, w, h) in rects:
        # if the width or height of the region is less than 10% of the
        # image width or height, ignore it (i.e., filter out small
        # objects that are likely false-positives)
        if w / float(W) > 0.2 or h / float(H) > 0.2:
            continue
        #if w / float(W) < 0.05 or h / float(H) < 0.05:
        #    continue
        # extract the region from the input image, convert it from BGR to
        # RGB channel ordering, and then resize it to 224x224 (the input
        # dimensions required by our pre-trained CNN)


        roi = image[y:y + h, x:x + w]

        # Crop, but add a border
        old_size = roi.shape[:2]
        ratio = float(299)/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])
        roi = cv2.resize(roi, (new_size[1], new_size[0]))

        delta_w = 398 - new_size[1] # this gets us back to the original image ratio size from taining
        delta_h = 299 - new_size[0]
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)

        color = [0, 0, 0]
        roi = cv2.copyMakeBorder(roi, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)

        roi = cv2.resize(roi, (299, 299))

        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        roi = img_to_array(roi)
        #roi = preprocess_input(roi)
        # update our proposals and bounding boxes lists
        proposals.append(roi)
        boxes.append((x, y, w, h))

    print("[INFO] {} boxes to invesitgate".format(len(boxes)))
    print("[INFO] {} proposals to predict".format(len(boxes)))
    
    # convert the proposals list into NumPy array and show its dimensions
    proposals = np.array(proposals, dtype="float32")
    print("[INFO] proposal shape: {}".format(proposals.shape))
    # classify each of the proposal ROIs using ResNet and then decode the
    # predictions
    print("[INFO] classifying proposals...")
    preds = model.predict(proposals)
    
    print("[INFO] {} predictions".format(len(preds)))
    labels = {"plane":[]}
    for (i, p) in enumerate(preds):
        # grab the prediction information for the current ROI
        #(imagenetID, label, prob) = p[0]
        prob = p[0]
        # filter out weak detections by ensuring the predicted probability
        # is greater than the minimum probability
        if prob >= MIN_CONF:
            # grab the bounding box associated with the prediction and
            # convert the coordinates
            (x, y, w, h) = boxes[i]
            box = (x, y, x + w, y + h)
            #print(box)
            # grab the list of predictions for the label and add the
            # bounding box + probability to the list
            #L = labels.get("plane",[])
            #L.append((box, prob))
            labels["plane"].append((box, prob))


    print("[INFO] {} predicted values".format(len(labels["plane"])))
    
    label="plane"
    # clone the original image so that we can draw on it
    print("[INFO] showing results for '{}'".format(label))
    
    # extract the bounding boxes and associated prediction
    # probabilities, then apply non-maxima suppression
    boxes = np.array([p[0] for p in labels[label]])
    proba = np.array([p[1] for p in labels[label]])
    if len(proba) > 0:
        idx = np.argmax(proba)
        maxBox=boxes[idx]
        (startX, startY, endX, endY) = maxBox
        x = startX
        y = startY
        w = endX - startX
        h = endY - startY
        floatX = x / W
        floatY = y / H
        floatW = w / W
        floatH = h / H
        return [floatX,floatY,floatW,floatH]
    else:
        return None



In [ ]:
session = fo.launch_app(dataset, auto=False)
dataset.exists("plane")


In [ ]:

#session.view = view

In [ ]:
view = dataset.skipped_view = dataset.match({"plane_spot": {"$exists": False, "$eq": None}}).match_tags("plane").shuffle()
for sample in view:
    box = findPlane(sample.filepath)
    if box != None:
        detections = []



        detections.append(fo.Detection(label="plane", bounding_box=box))

        # Store detections in a field name of your choice
        sample["plane_spot"] = fo.Detections(detections=detections)
        sample.save()

[INFO] 7365 regions found by selective search
[INFO] 6631 boxes to invesitgate
[INFO] 6631 proposals to predict
[INFO] proposal shape: (6631, 299, 299, 3)
[INFO] classifying proposals...
[INFO] 6631 predictions
[INFO] 15 predicted values
[INFO] showing results for 'plane'
[INFO] 8754 regions found by selective search
[INFO] 8055 boxes to invesitgate
[INFO] 8055 proposals to predict
[INFO] proposal shape: (8055, 299, 299, 3)
[INFO] classifying proposals...
[INFO] 8055 predictions
[INFO] 46 predicted values
[INFO] showing results for 'plane'
[INFO] 8240 regions found by selective search
[INFO] 7750 boxes to invesitgate
[INFO] 7750 proposals to predict
[INFO] proposal shape: (7750, 299, 299, 3)
[INFO] classifying proposals...
[INFO] 7750 predictions
[INFO] 9 predicted values
[INFO] showing results for 'plane'
[INFO] 8883 regions found by selective search
[INFO] 8416 boxes to invesitgate
[INFO] 8416 proposals to predict
[INFO] proposal shape: (8416, 299, 299, 3)
[INFO] classifying proposal

[INFO] 310 boxes to invesitgate
[INFO] 310 proposals to predict
[INFO] proposal shape: (310, 299, 299, 3)
[INFO] classifying proposals...
[INFO] 310 predictions
[INFO] 0 predicted values
[INFO] showing results for 'plane'
[INFO] 12817 regions found by selective search
[INFO] 11727 boxes to invesitgate
[INFO] 11727 proposals to predict
[INFO] proposal shape: (11727, 299, 299, 3)
[INFO] classifying proposals...
[INFO] 11727 predictions
[INFO] 60 predicted values
[INFO] showing results for 'plane'
[INFO] 8543 regions found by selective search
[INFO] 7969 boxes to invesitgate
[INFO] 7969 proposals to predict
[INFO] proposal shape: (7969, 299, 299, 3)
[INFO] classifying proposals...
[INFO] 7969 predictions
[INFO] 23 predicted values
[INFO] showing results for 'plane'
[INFO] 8536 regions found by selective search
[INFO] 7940 boxes to invesitgate
[INFO] 7940 proposals to predict
[INFO] proposal shape: (7940, 299, 299, 3)
[INFO] classifying proposals...
[INFO] 7940 predictions
[INFO] 2 predict

[INFO] 5023 boxes to invesitgate
[INFO] 5023 proposals to predict
[INFO] proposal shape: (5023, 299, 299, 3)
[INFO] classifying proposals...
[INFO] 5023 predictions
[INFO] 7 predicted values
[INFO] showing results for 'plane'
[INFO] 5459 regions found by selective search
[INFO] 4939 boxes to invesitgate
[INFO] 4939 proposals to predict
[INFO] proposal shape: (4939, 299, 299, 3)
[INFO] classifying proposals...
[INFO] 4939 predictions
[INFO] 2 predicted values
[INFO] showing results for 'plane'
[INFO] 11900 regions found by selective search
[INFO] 11038 boxes to invesitgate
[INFO] 11038 proposals to predict
[INFO] proposal shape: (11038, 299, 299, 3)
[INFO] classifying proposals...
[INFO] 11038 predictions
[INFO] 97 predicted values
[INFO] showing results for 'plane'
[INFO] 563 regions found by selective search
[INFO] 541 boxes to invesitgate
[INFO] 541 proposals to predict
[INFO] proposal shape: (541, 299, 299, 3)
[INFO] classifying proposals...
[INFO] 541 predictions
[INFO] 0 predicted

In [3]:
session = fo.launch_app(dataset, auto=False)


NameError: name 'fo' is not defined

In [ ]:
view = dataset.exists("plane_spot")
session.view = view

In [ ]:
# Create a view containing only the selected samples
selected_view = dataset.select(session.selected)

for sample in selected_view:
    sample.tags.append("relabel")
    sample.save()    